In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = 'data/'
train = pd.read_csv(data_path + 'train.csv', header = None)
test = pd.read_csv(data_path + 'test.csv', header = None)
trainLabels = pd.read_csv(data_path + 'trainLabels.csv', header = None)

In [3]:
print(train.shape)
print(test.shape)
print(trainLabels.shape)

(1000, 40)
(9000, 40)
(1000, 1)


In [4]:
svm = SVC()
svm.fit(train, trainLabels)
cross_val_score(svm, train, trainLabels, cv=5).mean()

0.9120000000000001

In [5]:
C = list(range(1, 11))
gamma = np.linspace(0.01, 0.5, 10)
param_grid = dict(C = C, gamma = gamma) 
grid_search = GridSearchCV(svm, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)
grid_result = grid_search.fit(train, trainLabels)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   12.0s finished


In [6]:
print(grid_result.best_score_)
print(grid_result.best_params_)

-0.085
{'C': 2, 'gamma': 0.01}


In [7]:
svm_para = SVC(C = 2, gamma = 0.01)
svm_para.fit(train, trainLabels)
cross_val_score(svm_para, train, trainLabels, cv=5).mean()

0.9180000000000001

In [8]:
y_pred = svm_para.predict(test)
#range(1, len(y_pred))
submit = pd.DataFrame(y_pred, index = range(1, len(y_pred)+1)).reset_index()
submit.columns = ['Id', 'Solution']
submit.head()

,Id,Solution
0,1,1
1,2,0
2,3,1
3,4,0
4,5,0


In [9]:
import csv
submit.to_csv(os.path.join(data_path, 'submit_Day_048.csv'), index = False)